In [34]:
from manifold3d import Manifold, CrossSection, set_circular_segments
import math
import numpy as np
set_circular_segments(32)

In [35]:
def generate_bit(type, diameter, length=50e-3):
    if type == "endmill":
        return Manifold.cylinder(length, diameter/2)
    elif type == "ballnose":
        shaft = Manifold.cylinder(length-diameter/2, diameter/2)
        bit = Manifold.sphere(diameter/2)
        return (shaft + bit).translate([0,0,diameter/2])
    elif type == "drill":
        conelen = 0.5*diameter/math.tan(math.radians(59))
        shaft = Manifold.cylinder(length-conelen, diameter/2)
        bit = Manifold.cylinder(conelen, diameter/2, 0).rotate(np.array([180, 0, 0]))
        return (shaft + bit).translate([0,0,conelen])

In [36]:
def generate_linear_cutarea(tool, dia, loc1, loc2, length=50e-3):
    delta = loc2 - loc1
    dcyl = 0 if tool == "endmill" else (0.5*dia if tool == "ballnose" else 0.5*dia/math.tan(math.radians(59)))
    
    dxy = (delta[0]**2 + delta[1]**2)**0.5
    dxyz = (delta[0]**2 + delta[1]**2 + delta[2]**2)**0.5
    phi = math.atan2(delta[1], delta[0])+np.pi
    theta = math.atan2(dxy, delta[2])

    mesh1 = generate_bit(tool, dia, length).rotate([0, 0, np.degrees(phi)]).translate(loc1)
    mesh2 = generate_bit(tool, dia, length).rotate([0, 0, np.degrees(phi)]).translate(loc2)

    cutarea = Manifold()
    if dxy > 1e-9:
        rectrod = CrossSection.square([dia, (length-dcyl)*np.sin(theta)]).translate([-0.5*dia, 0])
        cutarea += Manifold.extrude(rectrod, dxyz)\
            .warp(lambda v: [v[0], v[1], v[2]+v[1]/math.tan(theta)])\
            .rotate(np.array([np.rad2deg(theta), 0, 0]))\
            .translate([0,0,dcyl])
    if abs(delta[2]) > 1e-9:
        circ = CrossSection.circle(0.5*dia)
        cutarea += Manifold.extrude(circ, 1).scale([1,1,delta[2]])\
            .warp(lambda v: [v[0], v[1]-v[2]*math.tan(theta), v[2]])\
            .translate([0,0,dcyl])
    cutarea += Manifold.cylinder(length-dcyl, 0.5*dia).rotate([0, 0, np.degrees(phi)]).translate([0,0,dcyl])
    cutarea += Manifold.cylinder(length-dcyl, 0.5*dia).rotate([0, 0, np.degrees(phi)]).translate([0,-dxy,delta[2]+dcyl])
    if tool == "ballnose":
        circ = CrossSection.circle(0.5*dia)
        cutarea += Manifold.extrude(circ, dxyz)\
            .rotate(np.array([np.rad2deg(theta), 0, 0]))\
            .translate([0,0,dcyl])
        if dxy > 1e-9:
            cutarea += Manifold.revolve(circ, circular_segments=int(16*(np.pi-theta)/np.pi)+1, revolve_degrees=np.degrees(np.pi-theta))\
                .rotate(np.array([-90, 0., 90]))\
                .translate([0,0,dcyl])
            cutarea += Manifold.revolve(circ, circular_segments=int(16*(theta)/np.pi)+1, revolve_degrees=np.degrees(theta))\
                .rotate(np.array([-90, 0., -90]))\
                .translate([0,-dxy,dcyl+delta[2]])
        else:
            cutarea += Manifold.sphere(0.5*dia).translate([0,0,dcyl])

    cutarea = cutarea.rotate(np.array([0, 0, -np.rad2deg(phi)])).translate(loc1)

    return cutarea

mesha = generate_linear_cutarea("endmill", 4e-3, np.array([10e-3,10e-3,0e-3]), np.array([15e-3,10e-3,-5e-3]))

In [37]:
'''loc1 = np.array([0, 0, 0])
loc2 = np.array([10e-3, 10e-3, 5e-3])
tool = "ballnose"




mesh1 = generate_bit(tool, 4e-3)
mesh2 = generate_bit(tool, )

meshz = Manifold.compose([mesh]).rotate().translate(-loc1)
mesh2 = Manifold.compose([mesh]).translate(loc2)

cutarea = Manifold()
if dxy > 1e-9:
    rectrod = CrossSection.square([4e-3, (50e-3-dcyl)*np.sin(theta)]).translate([-2e-3, 0])
    meshz = meshz
    cutarea += Manifold.extrude(rectrod, dxyz)\
        .warp(lambda v: [v[0], v[1], v[2]+v[1]/math.tan(theta)])\
        .rotate(np.array([np.rad2deg(theta), 0, -np.rad2deg(phi)]))\
        .translate([0,0,dcyl])
if delta[2] > 1e-9:
    circ = CrossSection.circle(4e-3/2)
    cutarea += Manifold.extrude(circ, delta[2])\
        .warp(lambda v: [v[0], v[1]-v[2]*math.tan(theta), v[2]])\
        .rotate([-np.rad2deg(theta),0,0])\
        .rotate(np.array([np.rad2deg(theta), 0, -np.rad2deg(phi)]))\
        .translate([0,0,dcyl])
if tool == "ballnose":
    circ = CrossSection.circle(4e-3/2).translate([0,4e-3/2*np.sin(theta)])
    cutarea += Manifold.extrude(circ, delta[2])\
        .translate([0,0,4e-3/2*np.cos(theta)])\
        .rotate(np.array([np.rad2deg(theta), 0, -np.rad2deg(phi)]))
cutarea += meshz + mesh2'''

'loc1 = np.array([0, 0, 0])\nloc2 = np.array([10e-3, 10e-3, 5e-3])\ntool = "ballnose"\n\n\n\n\nmesh1 = generate_bit(tool, 4e-3)\nmesh2 = generate_bit(tool, )\n\nmeshz = Manifold.compose([mesh]).rotate().translate(-loc1)\nmesh2 = Manifold.compose([mesh]).translate(loc2)\n\ncutarea = Manifold()\nif dxy > 1e-9:\n    rectrod = CrossSection.square([4e-3, (50e-3-dcyl)*np.sin(theta)]).translate([-2e-3, 0])\n    meshz = meshz\n    cutarea += Manifold.extrude(rectrod, dxyz)        .warp(lambda v: [v[0], v[1], v[2]+v[1]/math.tan(theta)])        .rotate(np.array([np.rad2deg(theta), 0, -np.rad2deg(phi)]))        .translate([0,0,dcyl])\nif delta[2] > 1e-9:\n    circ = CrossSection.circle(4e-3/2)\n    cutarea += Manifold.extrude(circ, delta[2])        .warp(lambda v: [v[0], v[1]-v[2]*math.tan(theta), v[2]])        .rotate([-np.rad2deg(theta),0,0])        .rotate(np.array([np.rad2deg(theta), 0, -np.rad2deg(phi)]))        .translate([0,0,dcyl])\nif tool == "ballnose":\n    circ = CrossSection.circle(4

In [38]:
import polyscope as ps

ps.init()
ps.set_up_dir("z_up")
ps.set_front_dir("neg_y_front")
#ps.register_surface_mesh("mesh1", mesh1.to_mesh().vert_properties[:, :3], mesh1.to_mesh().tri_verts)
#ps.register_surface_mesh("mesh2", mesh2.to_mesh().vert_properties[:, :3], mesh2.to_mesh().tri_verts)
ps.register_surface_mesh("mesha", mesha.to_mesh().vert_properties[:, :3], mesha.to_mesh().tri_verts)
ps.show()